Analisis Segmentación de productos (ABC), (ABC - XYZ), Syntetos–Boylan, Croston, Análisis ABC multi-criterio

1️. Análisis ABC clásico (ventas o ingresos)

Qué hace: Clasifica productos en A, B, C según su contribución acumulada a ventas/ingresos.

Problemas que resuelve:
Falta de foco en productos clave: sin ABC se gestiona igual un SKU que aporta 50% de las ventas y otro que aporta 1%.
Recursos dispersos: ayuda a priorizar inventarios, atención de proveedores y esfuerzos comerciales.
Optimización de inventarios: asigna stock de seguridad diferenciado: más alto en A, moderado en B, mínimo en C.

In [11]:
import pandas as pd
import pandas as pd 
import matplotlib.pyplot as plt

df = pd.read_csv("supply_chain_data.csv")

#ABC Ingresos

abc = df.groupby("SKU").agg({
    "Number of products sold":"sum",
    "Revenue generated":"sum"
}).reset_index()

abc = abc.sort_values("Revenue generated", ascending=False)
abc["cumulative_share"]= abc["Revenue generated"].cumsum() / abc["Revenue generated"].sum()

def classify_abc(x):
    if x <= 0.8: return "A"
    elif x <= 0.95: return "B"
    else: return "C"

abc["ABC_class"] = abc["cumulative_share"].apply(classify_abc)
print(abc.head(20))

      SKU  Number of products sold  Revenue generated  cumulative_share  \
47  SKU51                      154        9866.465458          0.017082   
32  SKU38                      705        9692.318040          0.033862   
25  SKU31                      168        9655.135103          0.050578   
90  SKU90                      320        9592.633570          0.067185   
12   SKU2                        8        9577.749626          0.083767   
26  SKU32                      781        9571.550487          0.100338   
64  SKU67                      513        9473.798033          0.116740   
87  SKU88                      737        9444.742033          0.133092   
48  SKU52                      820        9435.762609          0.149428   
10  SKU18                      620        9364.673505          0.165641   
99  SKU99                      627        9185.185829          0.181543   
28  SKU34                      602        9061.710896          0.197231   
17  SKU24                

2. Análisis ABC–XYZ

Qué hace: Combina importancia (ABC) con variabilidad de la demanda (XYZ).

Problemas que resuelve:
Políticas de inventario genéricas: ajusta estrategias según variabilidad. Ejemplo:
AX: críticos y estables → reabastecimiento automático.
CZ: poco importantes y variables → compras bajo pedido.
Exceso de inventario en productos impredecibles: identifica dónde es más eficiente usar políticas pull.
Mejor pronóstico de demanda: detecta qué SKUs necesitan modelos avanzados por su variabilidad.

In [12]:
# Variabilidad de la demanda por SKU
df_group = df.groupby("SKU").agg({
    "Number of products sold":"sum",
    "Revenue generated":"sum",
    "Number of products sold":"std"
}).rename(columns={"Number of products sold":"std_demand"}).reset_index()

df_group["mean_demand"] = df.groupby("SKU")["Number of products sold"].mean().values
df_group["cv"] = df_group["std_demand"] / df_group["mean_demand"]

def classify_xyz(cv):
    if cv <= 0.5: return "X"   # estable
    elif cv <= 1.0: return "Y" # media variabilidad
    else: return "Z"           # muy variable

df_group["XYZ_class"] = df_group["cv"].apply(classify_xyz)

# Ahora combinas ABC + XYZ
abc_xyz = abc.merge(df_group[["SKU","XYZ_class"]], on="SKU", how="left")
abc_xyz["ABC_XYZ"] = abc_xyz["ABC_class"] + abc_xyz["XYZ_class"]

print(abc_xyz.head(20))


      SKU  Number of products sold  Revenue generated  cumulative_share  \
0   SKU51                      154        9866.465458          0.017082   
1   SKU38                      705        9692.318040          0.033862   
2   SKU31                      168        9655.135103          0.050578   
3   SKU90                      320        9592.633570          0.067185   
4    SKU2                        8        9577.749626          0.083767   
5   SKU32                      781        9571.550487          0.100338   
6   SKU67                      513        9473.798033          0.116740   
7   SKU88                      737        9444.742033          0.133092   
8   SKU52                      820        9435.762609          0.149428   
9   SKU18                      620        9364.673505          0.165641   
10  SKU99                      627        9185.185829          0.181543   
11  SKU34                      602        9061.710896          0.197231   
12  SKU24                

Análisis Croston / Syntetos–Boylan (demandas intermitentes)

Qué hace: Pronostica productos con ventas irregulares (muchos ceros).
Problemas que resuelve:

Errores en forecast de productos de baja rotación: los métodos clásicos (promedios, ARIMA) fallan con series intermitentes.
Sobreinventario por seguridad excesiva: sin estos modelos, se tiende a cubrir “por las dudas”.
Riesgo de desabasto en productos de demanda esporádica: da estimaciones más realistas del consumo.

| Clasificación                   | Condición                           |
| ------------------------------- | ----------------------------------- |
| **Liso (Smooth)**               | Alta frecuencia y baja variabilidad |
| **Errático (Erratic)**          | Alta frecuencia y alta variabilidad |
| **Intermitente (Intermittent)** | Baja frecuencia y baja variabilidad |
| **Lumpy (Irregular)**           | Baja frecuencia y alta variabilidad |


In [51]:
import pandas as pd
import numpy as np

# Cargar dataset
df = pd.read_csv("supply_chain_data.csv")

# Calcular métricas por SKU
classification = []

for sku in df["SKU"].unique():
    ts = df[df["SKU"] == sku]["Number of products sold"].fillna(0).values
    
    mean_demand = np.mean(ts)
    std_demand = np.std(ts)
    
    # Coeficiente de variación (CV)
    cv = std_demand / mean_demand if mean_demand > 0 else 0
    
    # Frecuencia de demanda (proporción de periodos con ventas > 0)
    freq = np.count_nonzero(ts) / len(ts)
    
    # Clasificación basada en umbrales
    if freq > 0.5 and cv <= 0.5:
        demand_type = "Liso (Smooth)"
    elif freq > 0.5 and cv > 0.5:
        demand_type = "Errático (Erratic)"
    elif freq <= 0.5 and cv <= 0.5:
        demand_type = "Intermitente (Intermittent)"
    else:
        demand_type = "Lumpy (Irregular)"
    
    classification.append({
        "SKU": sku,
        "Mean_demand": mean_demand,
        "Std_demand": std_demand,
        "CV": cv,
        "Frequency": freq,
        "Classification": demand_type
    })

# Convertir a DataFrame
class_df = pd.DataFrame(classification)
class_df



,SKU,Mean_demand,Std_demand,CV,Frequency,Classification
0,SKU7,426.0,0.0,0.0,1.0,Liso (Smooth)
1,SKU8,150.0,0.0,0.0,1.0,Liso (Smooth)
2,SKU17,126.0,0.0,0.0,1.0,Liso (Smooth)
3,SKU21,601.0,0.0,0.0,1.0,Liso (Smooth)
4,SKU23,391.0,0.0,0.0,1.0,Liso (Smooth)
...,...,...,...,...,...,...
95,SKU80,872.0,0.0,0.0,1.0,Liso (Smooth)
96,SKU82,336.0,0.0,0.0,1.0,Liso (Smooth)
97,SKU86,223.0,0.0,0.0,1.0,Liso (Smooth)
98,SKU90,320.0,0.0,0.0,1.0,Liso (Smooth)


In [57]:
for sku in df["SKU"].unique():
    ts = df[df["SKU"] == sku]["Number of products sold"].fillna(0).values
    print(sku, ts)

SKU7 [426]
SKU8 [150]
SKU17 [126]
SKU21 [601]
SKU23 [391]
SKU27 [352]
SKU28 [394]
SKU29 [253]
SKU33 [616]
SKU35 [449]
SKU38 [705]
SKU44 [919]
SKU49 [99]
SKU50 [633]
SKU59 [484]
SKU62 [270]
SKU71 [637]
SKU72 [478]
SKU73 [375]
SKU85 [25]
SKU88 [737]
SKU89 [134]
SKU91 [916]
SKU92 [276]
SKU94 [987]
SKU96 [324]
SKU0 [802]
SKU2 [8]
SKU5 [147]
SKU12 [336]
SKU18 [620]
SKU22 [884]
SKU24 [209]
SKU25 [142]
SKU26 [353]
SKU30 [327]
SKU43 [598]
SKU45 [24]
SKU46 [859]
SKU48 [29]
SKU51 [154]
SKU54 [622]
SKU55 [701]
SKU57 [227]
SKU61 [117]
SKU68 [163]
SKU70 [32]
SKU74 [904]
SKU76 [241]
SKU77 [359]
SKU78 [946]
SKU79 [198]
SKU81 [774]
SKU83 [663]
SKU84 [618]
SKU87 [79]
SKU93 [114]
SKU95 [672]
SKU97 [62]
SKU99 [627]
SKU1 [736]
SKU3 [83]
SKU4 [871]
SKU6 [65]
SKU9 [980]
SKU10 [996]
SKU11 [960]
SKU13 [249]
SKU14 [562]
SKU15 [469]
SKU16 [280]
SKU19 [187]
SKU20 [320]
SKU31 [168]
SKU32 [781]
SKU34 [602]
SKU36 [963]
SKU37 [963]
SKU39 [176]
SKU40 [933]
SKU41 [556]
SKU42 [155]
SKU47 [910]
SKU52 [820]
SKU53 [242]
S

Análisis ABC multi-criterio (ventas + ingresos, margen, criticidad)

Qué hace: Clasifica productos considerando más de una métrica.

Problemas que resuelve:
Visión parcial del portafolio: el ABC clásico basado solo en ingresos puede dejar fuera productos de alta rotación o estratégicos.
Desbalance entre ventas y rentabilidad: identifica productos con muchas unidades vendidas pero bajo margen, o pocos vendidos pero críticos por revenue.
Mala priorización estratégica: permite integrar criterios como margen, lead time, criticidad para el cliente.

In [15]:
from sklearn.preprocessing import MinMaxScaler

# Normalizamos ventas e ingresos
scaler = MinMaxScaler()
abc_multi = abc.copy()
abc_multi[["norm_sales","norm_revenue"]] = scaler.fit_transform(
    abc_multi[["Number of products sold","Revenue generated"]]
)

# Score combinado (50% ventas, 50% revenue)
abc_multi["score"] = 0.5*abc_multi["norm_sales"] + 0.5*abc_multi["norm_revenue"]
abc_multi = abc_multi.sort_values("score", ascending=False)

# Clasificación ABC sobre score
abc_multi["cumulative_share"] = abc_multi["score"].cumsum() / abc_multi["score"].sum()
abc_multi["ABC_multi"] = abc_multi["cumulative_share"].apply(classify_abc)

print(abc_multi.head(20))


      SKU  Number of products sold  Revenue generated  cumulative_share  \
48  SKU52                      820        9435.762609          0.017838   
94  SKU94                      987        7888.356547          0.035608   
98  SKU98                      913        8525.952560          0.053353   
26  SKU32                      781        9571.550487          0.070949   
79  SKU80                      872        8651.672683          0.088420   
30  SKU36                      963        7573.402458          0.105586   
87  SKU88                      737        9444.742033          0.122589   
32  SKU38                      705        9692.318040          0.139549   
0    SKU0                      802        8661.996792          0.156319   
39  SKU44                      919        7152.286049          0.172556   
42  SKU47                      910        7089.474250          0.188629   
10  SKU18                      620        9364.673505          0.204349   
99  SKU99                

In [55]:
import pandas as pd

# Exportar a Excel con varias hojas
with pd.ExcelWriter("Analisis_SupplyChain.xlsx", engine="openpyxl") as writer:
    abc.to_excel(writer, sheet_name="ABC_clasico", index=False)
    abc_xyz.to_excel(writer, sheet_name="ABC_XYZ", index=False)
    class_df.to_excel(writer, sheet_name="Clasificacion_demanda", index=False)
    abc_multi.to_excel(writer, sheet_name="ABC_multicriterio", index=False)

print("✅ Archivo 'Analisis_SupplyChain.xlsx' generado con éxito")

✅ Archivo 'Analisis_SupplyChain.xlsx' generado con éxito
